### **STEP 1 — Chargement du modèle LDA et des données d’entraînement**

- Importe les librairies nécessaires (zipfile, os, joblib, pandas, numpy, etc.).
- Dézippe le fichier `Q5_model_LDA_V3.zip` dans `/content`.
- Définit le dossier du modèle : `MODEL_DIR = "/content/Q5_model_LDA_V3"`.
- Charge :
  - le modèle LDA entraîné (`lda_model.joblib`),
  - le vectorizer `CountVectorizer` (`vectorizer.joblib`),
  - la matrice des topics par playlist (`playlist_topic_matrix.joblib`),
  - le DataFrame des playlists d’entraînement (`df_playlists.parquet`),
  - le DataFrame des relations playlist–morceau (`df_pl_tracks.parquet`).
- Affiche les shapes de `df_playlists` et `df_pl_tracks` pour vérifier que tout est bien chargé.


In [3]:
import zipfile
import os
import joblib
import pandas as pd
import numpy as np
import re
import random
from collections import defaultdict

# =========================
# 1) Charger le modèle LDA + data TRAIN
# =========================

zip_path = "/content/Q5_model_LDA_V3.zip"

with zipfile.ZipFile(zip_path, 'r') as z:
    z.extractall("/content")

MODEL_DIR = "/content/Q5_model_LDA_V3"


lda = joblib.load(os.path.join(MODEL_DIR, "lda_model.joblib"))
vectorizer = joblib.load(os.path.join(MODEL_DIR, "vectorizer.joblib"))
playlist_topic_matrix = joblib.load(os.path.join(MODEL_DIR, "playlist_topic_matrix.joblib"))

df_playlists = pd.read_parquet(os.path.join(MODEL_DIR, "df_playlists.parquet"))   # TRAIN playlists
df_pl_tracks = pd.read_parquet(os.path.join(MODEL_DIR, "df_pl_tracks.parquet"))   # mapping pid -> tracks

print("TRAIN shapes :", df_playlists.shape, df_pl_tracks.shape)

TRAIN shapes : (50000, 3) (3353026, 5)


### **STEP 2 — Charger le jeu de TEST (slice mise de côté)**

- Charge le fichier `test_slices.parquet` qui contient les playlists réservées pour l’évaluation.
- Ce dataset n’a **jamais** été utilisé pour entraîner le modèle, donc il sert à mesurer la performance réelle.
- Affiche sa shape pour vérifier que les playlists test sont bien chargées.


In [5]:
# =========================
# 2) Charger le jeu de TEST (slice mise de côté)
# =========================

df_playlists_test = pd.read_parquet("/content/Q5_model_LDA_V3/test_slices.parquet")
print("Shape df_playlists_test :", df_playlists_test.shape)

Shape df_playlists_test : (1000, 3)


### **STEP 3 — Construire les mappings tracks (TRAIN / TEST)**

- Regroupe `df_pl_tracks` par `pid` pour obtenir, pour chaque playlist, l’ensemble des `track_uri`.
- Sépare ces mappings en deux parties :
  - `pl2tracks_train` : playlists utilisées pour l’entraînement.
  - `pl2tracks_test` : playlists utilisées uniquement pour l’évaluation.
- Crée également `track_meta`, un petit tableau utile pour afficher le nom et l’artiste d’un morceau à partir de son `track_uri`.


In [6]:
# =========================
# 3) Construire les mappings tracks (TRAIN / TEST)
# =========================

# mapping global pid -> set(track_uri)
pl2tracks_all = df_pl_tracks.groupby("pid")["track_uri"].apply(lambda s: set(s)).to_dict()

train_pids = set(df_playlists["pid"])
test_pids  = set(df_playlists_test["pid"])

# mapping TRAIN : pid (train) -> tracks
pl2tracks_train = {pid: tracks for pid, tracks in pl2tracks_all.items() if pid in train_pids}
# mapping TEST : pid (test) -> tracks
pl2tracks_test  = {pid: tracks for pid, tracks in pl2tracks_all.items() if pid in test_pids}

# meta par track_uri (pour lisibilité éventuelle)
track_meta = (
    df_pl_tracks
      .sort_values("track_name")
      .drop_duplicates("track_uri")[["track_uri", "track_name", "artist_name"]]
)

### **STEP 4 — Normalisation des vecteurs de topics (TRAIN)**

- Normalise chaque vecteur de topics de `playlist_topic_matrix` pour qu’il ait une norme 1.
- Cette normalisation permet de calculer la similarité cosinus entre playlists.
- `playlist_norm` est donc la version normalisée, utilisée plus tard pour trouver les playlists voisines.


In [7]:
# =========================
# 4) Normalisation des topics TRAIN pour cosinus
# =========================

playlist_norm = playlist_topic_matrix / (
    np.linalg.norm(playlist_topic_matrix, axis=1, keepdims=True) + 1e-12
)


### **STEP 5 — Calcul des topics pour les playlists de TEST**

- Nettoie le texte des playlists de test en appliquant la même fonction `clean_text` que pour le TRAIN.
- Transforme ces textes en vecteurs Bag-of-Words **avec le vectorizer entraîné sur le TRAIN**  
  (important : on n'appelle **pas** `fit`, seulement `transform`).
- Projette ces vecteurs BoW dans l’espace des topics grâce au modèle LDA entraîné.
- Le résultat `playlist_topic_test` contient un vecteur de topics pour chaque playlist de TEST.


In [8]:
# =========================
# 5) Topics pour les playlists de TEST
# =========================

def clean_text(text: str) -> str:
    text = text.lower()
    text = re.sub(r"[^a-z0-9]+", " ", text)
    return text

df_playlists_test["clean_text"] = df_playlists_test["text"].astype(str).apply(clean_text)

# BoW sur le TEST (vectorizer entraîné sur le TRAIN)
X_bow_test = vectorizer.transform(df_playlists_test["clean_text"])

# Projeter dans l'espace LDA (inférence)
playlist_topic_test = lda.transform(X_bow_test)
print("Shape playlist_topic_test :", playlist_topic_test.shape)

Shape playlist_topic_test : (1000, 30)


### **STEP 6 — Recherche des voisins et génération des recommandations (TEST)**

- `top_k_neighbors_for_query_theta` :
  - prend le vecteur de topics d’une playlist TEST (`theta_query`),
  - calcule sa similarité cosinus avec **toutes les playlists TRAIN**,
  - retourne les `k` playlists les plus similaires.

- `recommend_tracks_for_test_pid_with_seeds` :
  - récupère le vecteur LDA de la playlist TEST (déjà calculé à l'étape précédente),
  - trouve ses `k` playlists voisines dans le TRAIN,
  - agrège les morceaux présents dans ces playlists voisines,
  - exclut les morceaux déjà présents dans les seed tracks,
  - renvoie les 500 recommandations les mieux scorées.


Ces fonctions permettent d’effectuer des recommandations pour **une playlist de TEST**, en s’appuyant uniquement sur les playlists d’entraînement.


In [9]:
# =========================
# 6) Voisins & reco pour une playlist TEST (via θ_test)
# =========================

def top_k_neighbors_for_query_theta(theta_query, k=500):
    """
    theta_query : vecteur topics (1D) d'une playlist TEST
    Retourne des voisins parmi les playlists TRAIN (df_playlists, playlist_norm)
    """
    q = theta_query.reshape(1, -1)
    q_norm = q / (np.linalg.norm(q, axis=1, keepdims=True) + 1e-12)
    sims = (q_norm @ playlist_norm.T).ravel()  # similarité avec TOUTES les playlists TRAIN

    nn = np.argpartition(-sims, k)[:k]
    nn = nn[np.argsort(-sims[nn])]
    return nn, sims[nn]

def recommend_tracks_for_test_pid_with_seeds(query_pid, seed_tracks, topn=500, k_neighbors=500):
    """
    Recommandations pour une playlist de TEST :
    - on utilise les topics calculés pour le TEST comme requête
    - on va chercher des voisins dans le TRAIN
    - on agrège les tracks des voisins TRAIN (en excluant les seed_tracks)
    """
    # index de la playlist dans df_playlists_test
    rows = df_playlists_test.index[df_playlists_test["pid"] == query_pid]
    if len(rows) == 0:
        raise ValueError(f"PID {query_pid} pas trouvé dans df_playlists_test")
    pl_idx_test = int(rows[0])

    theta_q = playlist_topic_test[pl_idx_test]

    # voisins côté TRAIN
    nn_idx, nn_sims = top_k_neighbors_for_query_theta(theta_q, k=k_neighbors)
    nn_pids = df_playlists.iloc[nn_idx]["pid"].to_numpy()  # voisins = TRAIN

    query_tracks = set(seed_tracks)

    scores = defaultdict(float)
    for pid, s in zip(nn_pids, nn_sims):
        for t in pl2tracks_train.get(pid, ()):  # tracks des PLAYLISTS TRAIN
            if t not in query_tracks:
                scores[t] += float(s)

    items = sorted(scores.items(), key=lambda x: x[1], reverse=True)[:topn]
    recs = pd.DataFrame(items, columns=["track_uri", "score"])
    recs = recs.merge(track_meta, on="track_uri", how="left")
    return recs[["track_uri", "track_name", "artist_name", "score"]]

### **STEP 7 — Construction des seeds et définition de la métrique Recall@500 (TEST uniquement)**

- Définit `K = 25`, le nombre de morceaux conservés comme **seed tracks** (les seuls connus de la playlist côté utilisateur).
- Sélectionne les playlists de TEST ayant au moins `K + 1` morceaux, pour pouvoir séparer seed et ground truth.
- Tire aléatoirement un échantillon de playlists de TEST (jusqu’à 1000) pour l’évaluation.
- Fonction `make_seed_and_ground_truth_test` :
  - tire `K` morceaux comme **seeds**,
  - utilise le reste comme **ground truth** (GT), c’est-à-dire les morceaux que le modèle doit retrouver.
- Fonction `recall_at_500` :
  - calcule le **Recall@500**, soit la proportion de GT retrouvée dans les 500 recommandations retournées par le modèle.


In [10]:

# =========================
# 7) Construction des seeds & métrique Recall@500 (TEST uniquement)
# =========================

K = 25  # nombre de seed tracks

# playlists de TEST avec au moins K+1 morceaux
eval_pids_test = [
    pid for pid, tracks in pl2tracks_test.items()
    if len(tracks) > K + 1
]

print("Nombre de playlists TEST éligibles :", len(eval_pids_test))

random.seed(42)
eval_pids_sample_test = random.sample(eval_pids_test, k=min(1000, len(eval_pids_test)))
print("Nombre de playlists TEST utilisées pour l'évaluation :", len(eval_pids_sample_test))

def make_seed_and_ground_truth_test(pid, K):
    tracks = list(pl2tracks_test[pid])
    if len(tracks) <= K:
        return None, None
    seed_tracks = set(random.sample(tracks, K))
    gt_tracks = set(tracks) - seed_tracks
    return seed_tracks, gt_tracks

def recall_at_500(gt_tracks, rec_uris):
    """
    Recall@500 = (# de tracks de la ground truth retrouvés dans le top 500)
                 / (# de tracks dans la ground truth)
    """
    gt = set(gt_tracks)
    if not gt:
        return 0.0
    rec = set(rec_uris[:500])
    return len(gt & rec) / len(gt)

Nombre de playlists TEST éligibles : 733
Nombre de playlists TEST utilisées pour l'évaluation : 733


### **STEP 8 — Boucle d’évaluation sur le TEST (≈ 3 minutes)**

- Parcourt chaque playlist sélectionnée pour l’évaluation.
- Pour chaque playlist de TEST :
  1. Génère ses **seed tracks** et sa **ground truth**.
  2. Appelle la fonction de recommandation basée sur les **topics TEST** et les **voisins TRAIN**.
  3. Récupère les 500 recommandations produites.
  4. Calcule le **Recall@500** en comparant ces recommandations à la ground truth.
- Stocke la valeur du Recall pour chaque playlist.
- Affiche :
  - le nombre total de playlists évaluées,
  - la moyenne du Recall@500,
  - la médiane du Recall@500.


Ce STEP fournit un indicateur global de performance du modèle LDA + k-NN sur le **jeu de TEST**, totalement séparé du train.


In [11]:

# =========================
# 8) Boucle d'évaluation sur le TEST (3' pour run)
# =========================

recalls_test = []

for pid in eval_pids_sample_test:
    seed_tracks, gt_tracks = make_seed_and_ground_truth_test(pid, K)
    if not seed_tracks or not gt_tracks:
        continue

    recs = recommend_tracks_for_test_pid_with_seeds(
        pid,
        seed_tracks,
        topn=500,
        k_neighbors=500
    )
    rec_uris = recs["track_uri"].tolist()

    r = recall_at_500(gt_tracks, rec_uris)
    recalls_test.append(r)

print("Nombre de playlists TEST évaluées :", len(recalls_test))
print("Recall@500 TEST moyen :", float(np.mean(recalls_test)))
print("Recall@500 TEST médian :", float(np.median(recalls_test)))


Nombre de playlists TEST évaluées : 733
Recall@500 TEST moyen : 0.1225136185388589
Recall@500 TEST médian : 0.08
